In [1]:
import pandas as pd

In [2]:
df = pd.read_parquet("../data/yellow_tripdata_2023-01.parquet")

In [3]:
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.30,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.90,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.90,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.10,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.40,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3066761,2,2023-01-31 23:58:34,2023-02-01 00:12:33,NaN,3.05,NaN,None,107,48,0,15.80,0.00,0.5,3.96,0.0,1.0,23.76,NaN,NaN
3066762,2,2023-01-31 23:31:09,2023-01-31 23:50:36,NaN,5.80,NaN,None,112,75,0,22.43,0.00,0.5,2.64,0.0,1.0,29.07,NaN,NaN
3066763,2,2023-01-31 23:01:05,2023-01-31 23:25:36,NaN,4.67,NaN,None,114,239,0,17.61,0.00,0.5,5.32,0.0,1.0,26.93,NaN,NaN
3066764,2,2023-01-31 23:40:00,2023-01-31 23:53:00,NaN,3.15,NaN,None,230,79,0,18.15,0.00,0.5,4.43,0.0,1.0,26.58,NaN,NaN


Q1. Downloading the data: How many columns are there?

- 16
- 17
- 18
- 19

A: 19

In [4]:
df['duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds().div(60)
df['duration'].std().round(2)

42.59

Q2. Computing duration: What's the standard deviation of the trips duration in January?

- 32.59
- 42.59
- 52.59
- 62.59

A: 42.59

In [5]:
df_cleaned = df.query("1 <= duration <= 60")

In [6]:
round(len(df_cleaned) / len(df), 2)

0.98

Q3. Dropping outliers: What fraction of the records left after you dropped the outliers?

- 90%
- 92%
- 95%
- 98%

A: 98%

In [7]:
from sklearn.feature_extraction import DictVectorizer

vectorizer = DictVectorizer()

location_ids = df_cleaned[['PULocationID', 'DOLocationID']].astype(str)
location_ids = location_ids.to_dict(orient = "records")

feature_matrix = vectorizer.fit_transform(location_ids)

In [8]:
feature_matrix.shape

(3009173, 515)

Q4. One-hot encoding: What's the dimensionality of this matrix (number of columns)?

- 2
- 155
- 345
- 515
- 715

A: 515

In [13]:
feature_matrix

<3009173x515 sparse matrix of type '<class 'numpy.float64'>'
	with 6018346 stored elements in Compressed Sparse Row format>

In [11]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

train_x = feature_matrix.copy()
train_y = df_cleaned.duration

model = LinearRegression()
model.fit(train_x, train_y)

y_score_train = model.predict(train_x)

print(f"RMSE train={root_mean_squared_error(train_y, y_score_train)}")

RMSE train=7.649262060514987


Q5. Training a model: What's the RMSE on train?

- 3.64
- 7.64
- 11.64
- 16.64

A: 7.64

In [18]:
df_eval = pd.read_parquet("../data/yellow_tripdata_2023-02.parquet")
df_eval['duration'] = (df_eval['tpep_dropoff_datetime'] - df_eval['tpep_pickup_datetime']).dt.total_seconds().div(60)
df_eval = df_eval.query("1 <= duration <= 60")

location_ids_eval = df_eval[['PULocationID', 'DOLocationID']].astype(str)
location_ids_eval = location_ids_eval.to_dict(orient = "records")

eval_x = vectorizer.transform(location_ids_eval)
eval_y = df_eval.duration

y_score_eval = model.predict(eval_x)
print(f"RMSE eval={root_mean_squared_error(eval_y, y_score_eval)}")

RMSE eval=13.323885290924599


Q6. Evaluating the model: What's the RMSE on validation?

- 3.81
- 7.81
- 11.81
- 16.81

A: ?